In [1]:
!nvidia-smi

Wed Aug 12 01:28:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.100      Driver Version: 440.100      CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce 920MX       Off  | 00000000:03:00.0 Off |                  N/A |
| N/A   44C    P0    N/A /  N/A |      5MiB /  2004MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# !git clone https://github.com/ShivamShrirao/dnn_from_scratch.git

In [3]:
# !git clone https://github.com/ShivamShrirao/deep_Q_learning.git

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# %cd deep_Q_learning

In [1]:
import gym
import time
import cv2
import numpy as np

from settings import *
from agent import *
from experience import *
from atari_wrappers import *

In [7]:
agt = Agent(actions=[0,2,3], epsilon=1, min_epsilon=0.1, eps_decay=1e-5)
D_exp = ReplayMemory(capacity=100000)

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- InputLayer(InputLayer)  (None, 80, 80, 4)          echo             0
__________________________________________________________________________________________
0 Conv2D(Conv2D)          (None, 40, 40, 32)         relu             1184
__________________________________________________________________________________________
1 Conv2D(Conv2D)          (None, 20, 20, 64)         relu             18496
__________________________________________________________________________________________
2 Conv2D(Conv2D)          (None, 10, 10, 128)        relu             73856
__________________________________________________________________________________________
3 Flatten(Flatten)        (None, 12800)              echo             0
__________________________________________________________________________________________
4 Den

In [8]:
env = gym.make('Pong-v0')
env = FrameStack(env, NFRAMES)      # preprocess and stack frames

In [11]:
for i_episode in range(2):
    observation = env.reset()
    ep_score = 0
    start = time.time()
    for t in range(10000):
#         env.render()
        action = agt.get_action(observation)
        next_observation, reward, done, info = env.step(action)
        ep_score += reward

        D_exp.store_transition(observation, agt.actions.index(action), reward, next_observation, done)
        observation = next_observation

        if len(D_exp.ndone) > BATCH_SIZE:
            agt.train(D_exp)
        
        if done:
            break
        print('\r', t, ep_score, end='  ')
    print(f"\rEpisode {i_episode+1} finished after {t+1} timesteps, Score: {ep_score}, Epsilon: {agt.epsilon}, Time: {time.time()-start}")
#     agt.model.save_weights("model.w8s")
env.close()

 1433 -18.0  
Episode 0 finished after 1435 timesteps, Score: -19.0, Epsilon: 0.9738200000001191, Time: 260.87434434890747
 1371 -19.0  
Episode 1 finished after 1373 timesteps, Score: -20.0, Epsilon: 0.9600900000001816, Time: 247.1152000427246


In [11]:
env.close()